# Project Title
### Data Engineering Capstone Project

#### Project Summary
As a data engineer, We need to decide what would be the best approach for your company in terms of selecting tools, cost /budget, scalability of data, end user, data model, volumetric and so on

First I have chosen Udacity provided dataset, added to that picked some inter-related datasets such as Airline details, Visa Details, Cost of living

So now am very clear with my data and end users, Definitely this might be used by US Immigration and inter related teams.

Coming to possible approaches, 

###### Option 1. Python, S3 for Storage, Spark for Data Processing (Eventually for Big Data), Airflow for scheduling pipelines
		  Data Model    : Star Schema
		  Cost Involve  : Yes
		  Maintenance   : Easy
		  Scalability   : Yes
		  Long Run      : Yes
		  Over all Process: Stage the data into redshift tables as same as source then after processing, can load into Redshift tables, schedule it using airflow
		  

###### Option 2. Python, on-premise drives, Spark / Pandas, Juypter notebook 
		  Data Model    : Star Schema
		  Cost Involve  : No
		  Maintenance   : Difficult
		  Scalability   : No
		  Long Run      : Yes
		  Over all Process: Process the data(Filtering, Cleansing, Transformation) using Spark then after processing, can save the data as Parquet files for Data Analysis
		  		  
I would like / suggest to go for option 1 since it's very reliable approach in terms of scalability, maintenance and long run
but dropping off since Apache airflow environment does not have access to US Immigration data(6GB) and Project Workspace does not have Apache airflow.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

### Step 1: Scope the Project and Gather Data

In this project, we have collected data from different sources such as Immigration, Temperature, Demographics, Airline and created the data model based on data assessment then performed transformation, filtering, cleansing and finally saved as Parquets file.

It would help data analytics team to get answer following questions

###### Tourism Dept Questions:
1. What is the average tourist count on each month?
2. Which is the most popular cities in US among tourister?
3. What is the age range of tourister?

###### Immigration Dept Questions:
1. What are the top airports used by immigrants while arrival and departure?
2. What are the preferred cities of immigrants by considering rent index?
3. What are the preferred cities of immigrants by considering average temperature?

###### Airline Dept Questions:
1. What are the top airlines used by Immigrants?
2. What are the top visa categories of immigrants?
3. What is the age range of travellers by visa type?

###### Demographic Dept Questions:
1. How much percentage of immigrants adding by race category?
2. What is the percentage of gender equality?
3. What is the percenatge of Foreign born?  

###### Metrological Dept Questions:
1. What would be the temperture of top 5 US cities used by Immigrants in 2030? - Forecasting
2. What is the temperature difference of top 5 US cities preferred by business Immigrants and their orgin?
3. What are the least 5 US cities of Immigrants and its temperature?


#### Describe and Gather Data 
| Dataset                    | Format   | Description                                                                                                                                          | Source Link                                                                        |   |
|----------------------------|----------|------------------------------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------|---|
| I94 Immigration Data       | sas7bdat | This data comes from the US National Tourism and Trade Office                                                                                        | https://travel.trade.gov/research/reports/i94/historical/2016.html                 |   |
| World Temperature Data     | CSV      | Global Land and Ocean-and-Land Temperatures                                                                                                          | https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data |   |
| U.S. City Demographic Data | CSV      | This dataset contains information about the demographics of all US cities and census-designated places with a population greater or equal to 65,000. | https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/     |   |
| Airport Code Table         | CSV      | This is a simple table of airport codes and corresponding cities                                                                                     | https://datahub.io/core/airport-codes#data                                         |   |
| Visa Code                  | CSV      | Describe the classes of admission                                                                                                                    | https://www.dhs.gov/immigration-statistics/nonimmigrant/NonimmigrantCOA            |   |
| Airline Code               | CSV      | Describe Major Airline , Country, Carrier code,                                                                                                      | https://www.kwe.co.jp/en/useful-contents/code1                                     |   |
| Cost of Living             | CSV      | Details of US States Cost of Living categorized by Rent, Groceries, Restaurants, Local Purchasing Power                                              | https://www.numbeo.com/cost-of-living/country_result.jsp?country=United+States     |   |                                    |   |

In [1]:
# Import all necessary packages, some with alias name for easy access
import pandas as pd
import os
import glob
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,substring_index

In [2]:
# Entry to spark application, connect the cluster with an application
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

In [3]:
# Preprocessing of Immigration Data
# Convert whole SAS Data into Parquet file as it would be easy for processing in spark
# Preprocessing of whole 2016 data (80Million records) will take 10 - 15 mins of time
filepath='../../data/'
all_files = []
for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.sas7bdat'))
        for f in files :
            all_files.append(os.path.abspath(f))

#Comment out all_files list to process whole 2016 data           
all_files = ['/data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat', '/data/18-83510-I94-Data-2016/i94_sep16_sub.sas7bdat']

for i in all_files:
    df_spark =spark.read.format('com.github.saurfang.sas.spark').load(i)
    df_spark = df_spark.union(df_spark)

df_spark.write.mode("overwrite").parquet("stage")
print("done")
    

done


In [4]:
#Print Schema of the immigration dataset
df_spark.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

### Step 2: Explore and Assess the Data
1. **Immigration Dataset**
    * Convert the datatype to integer for cicid,i94yr,i94mon,i94cit,i94res,BIRYEAR
    * Transform i94mode as Air, Sea, Land and Not Reported according to the values else categories as 'NA'
    * Get the city and resident name from country table
    * Convert the SAS date to Readable data format for Arrival and Departure date
    * Transform i94visa as Business, Pleasure and Student according to the values otherwise catgeories as 'NA'
    * Convert the string type to date for DTADDTO
    * Remove duplicates
    * Rename all the columns with proper meaningful name and avoid space or special characters


2. **Demographics Dataset**
    * Transpose the race row values into columns, the New columns are Whitecount, Asiancount, HispanicOrLatinocount, BlackOrAfricanAmerican, AmericanIndianandAlaskaNative
    * Convert the float dataype to integer for Male Population, Female Population, Total Population, Number of Veterans, Foreign-born, Average Household Size
    * Remove duplicates
    * Rename all the columns with proper meaningful name and avoid space or special characters


3. **Temperature Dataset**
    * Remove records having NULL temperature and NULL Dates
    * Convert datatype timestamp to date for dt
    * Remove duplicates
   

4. **Airport Dataset**
    * Remove records where state='closed'
    * Filter only US airports
    * Remove duplicates
    * Derive State code from iso_region
    

5. **Cost of living Dataset**
    * Split the city and city code comma separated values into two
    * Remove duplicates 

6. **Countries**
    * Country dataset is derived from I94_SAS_Labels_Descriptions.SAS
    * Columns are segregated with delimters =
    * Flag column identify valid and invalid records

7. **Port**
    * Port dataset is derived from I94_SAS_Labels_Descriptions.SAS
    * Columns are segregated with delimters ,
    * Flag column identify valid and invalid records
    
    
8. **Airline**
    * No correction in Airline dataset


9. **Visa**
    * No correction in Visa dataset
    

In [5]:
#Reload the immigrants dataset from stage folder
filepath='stage/*.parquet'
df_immigrants = spark.read.parquet(filepath)

#Eliminate unwanted columns
df_immigrants = df_immigrants.drop('count','DTADFILE','VISAPOST','OCCUP','ENTDEPA','ENTDEPD','ENTDEPU','MATFLAG','insnum','admnum')
print("Total Immigration Dataset Count", df_immigrants.count())

Total Immigration Dataset Count 7467572


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
When designing the data model, we should study the dataset first,  In our case the dataset are Immigration, temperature, demographics, Airport and supporting data's and all it has state,country as common.

As i said earlier, my end user would be Immigration team and they want to analyse immigration key metrics and related data. so i have chosen star model, Immigration dataset act as fact table and rest act as dimension tables


#### 3.2 Mapping Out Data Pipelines
![Schema](ConceptualDataModel.jpg)

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [6]:
#Countries
df_countries = spark.read.csv('Countries.csv',header=True,sep=',',inferSchema='true')
df_countries.createOrReplaceTempView("countries")
df_countries.dropDuplicates()
df_countries.printSchema()

root
 |-- CountryCode: integer (nullable = true)
 |-- CountryName: string (nullable = true)
 |-- Flag: string (nullable = true)



In [7]:
#Port
df_port = spark.read.csv('ports.csv',header=True,sep=',',inferSchema='true')
df_port.createOrReplaceTempView("port")
df_port.dropDuplicates()
df_port.printSchema()

root
 |-- PortEntry: string (nullable = true)
 |-- PortName: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Flag: string (nullable = true)



In [8]:
#Visa
df_visa = spark.read.csv('Visa Type.csv',header=True,sep=',',inferSchema='true')
df_visa.createOrReplaceTempView("visa")
df_visa.dropDuplicates()
df_visa.printSchema()

root
 |-- VisaCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Category: string (nullable = true)



In [9]:
#Airline
df_airline = spark.read.csv('Airline.csv',header=True,sep=',',inferSchema='true')
df_airline.createOrReplaceTempView("Airline")
df_airline.dropDuplicates()
df_airline.printSchema()

root
 |-- AirlineCode: integer (nullable = true)
 |-- Airline: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- CarrierCode2: string (nullable = true)
 |-- CarrierCode3: string (nullable = true)



In [10]:
#Immigration
df_immigrants.createOrReplaceTempView("immigrants")
immigrants = spark.sql("""
                            SELECT distinct 
                            int(i.cicid) as ID,
                            int(i94yr) as ReportingYear,
                            int(i94mon) as ReportingMonth,
                            int(i.i94cit) as CityCode,
                            int(i.i94res) as ResidentCode,
                            cit.CountryName as City,
                            res.CountryName as Resident,
                            i.i94port as PortEntry,
                            case when i.i94mode=1 then 'Air'
                                 when i.i94mode=2 then 'Sea'
                                 when i.i94mode=3 then 'Land'
                                 when i.i94mode=9 then 'Not Reported'
                                 Else 'NA'
                            End as ModeofTravel,
                            date_add('1960-01-01',arrdate) as ArrivalDate,
                            date_add('1960-01-01',depdate) as DepartureDate,
                            i94addr as Statecode,
                            i94bir as Age,
                            case when i.i94visa=1 then 'Business'
                                 when i.i94visa=2 then 'Pleasure'
                                 when i.i94visa=3 then 'Student'
                                 Else 'NA'
                            End as VisaType,
                            visatype as VisaCode,
                            int(BIRYEAR) as BirthYear,
                            to_date(DTADDTO, 'MMddyyyy') as ValidTill,
                            Gender,
                            i.airline as CarrierCode2,
                            fltno as FlightNumber
                            FROM immigrants i 
                            join countries cit on int(i.i94cit)=cit.CountryCode and cit.flag='Valid'
                            join countries res on int(i.i94res)=res.CountryCode and cit.flag='Valid'
                        """)
print("Done")

Done


In [11]:
#Print the new modifed immigration schema
print("Processing...")
print("Total Valid immigrants",immigrants.count())

Processing...


3254348

In [12]:
immigrants.show(1)

+------+-------------+--------------+--------+------------+-------+--------+---------+------------+-----------+-------------+---------+----+--------+--------+---------+----------+------+------------+------------+
|    ID|ReportingYear|ReportingMonth|CityCode|ResidentCode|   City|Resident|PortEntry|ModeofTravel|ArrivalDate|DepartureDate|Statecode| Age|VisaType|VisaCode|BirthYear| ValidTill|Gender|CarrierCode2|FlightNumber|
+------+-------------+--------------+--------+------------+-------+--------+---------+------------+-----------+-------------+---------+----+--------+--------+---------+----------+------+------------+------------+
|864993|         2016|             9|     108|         108|DENMARK| DENMARK|      FTL|         Air| 2016-09-04|   2016-09-05|       FL|53.0|Pleasure|      WT|     1963|2016-12-02|     F|          DY|        7041|
+------+-------------+--------------+--------+------------+-------+--------+---------+------------+-----------+-------------+---------+----+--------

In [13]:
#Demographics
df_demographics = spark.read.csv('us-cities-demographics.csv',header=True,sep=';',inferSchema='true')
print("Original File Count",df_demographics.count())

Original File Count 2891


In [14]:
df_demographics.createOrReplaceTempView("demographics")
demographics = spark.sql("""Select temp.State, 
                                State_Code as StateCode,
                                temp.City,
                                Avg(Median_Age) as MedianAge,
                                int(Avg(Male_Population)) as MalePopulation,
                                int(Avg(Female_Population)) as FemalePopulation,
                                int(Avg(Total_Population)) as TotalPopulation,
                                int(Avg(NumberofVeterans)) as NumberofVeterans, 
                                int(Avg(Foreign_born)) as ForeignBorn, 
                                int(Avg(Average_Household_Size)) as AverageHouseholdSize, 
                                Sum(Whitecount) as White,
                                Sum(Asiancount) as Asian, 
                                Sum(HispanicOrLatinocount) as HispanicOrLatino ,
                                Sum(AmericanIndianandAlaskaNative) as AmericanIndianandAlaskaNative,
                                Sum(BlackOrAfricanAmerican) as BlackOrAfricanAmerican
                            from
                            (SELECT state,city,`Median Age` as Median_Age
                            ,`Male Population` as Male_Population,`Female Population` as Female_Population
                            ,`Total Population` as Total_Population
                            ,`Number of Veterans` as NumberofVeterans
                            ,`Foreign-born` as Foreign_born
                            ,`Average Household Size` as Average_Household_Size
                            ,`State Code` as State_Code
                            ,case when race='White' then count else 0
                             end as Whitecount
                             ,case when race='Asian' then count else 0
                             end as Asiancount
                             ,case when race='Hispanic or Latino' then count else 0
                             end as HispanicOrLatinocount
                             ,case when race='Black or African-American' then count else 0
                             end as BlackOrAfricanAmerican
                             ,case when race='American Indian and Alaska Native' then count else 0
                             end as AmericanIndianandAlaskaNative
                            FROM demographics ) as temp
                            group by temp.state,temp.city,temp.Median_Age,temp.Male_Population,temp.Female_Population
                            ,temp.Total_Population,temp.NumberofVeterans ,temp.Foreign_born,temp.Average_Household_Size
                            ,temp.State_Code
                            """)
print("Total Record count after transpose race into columns", demographics.count())

Total Record count after transpose race into columns 596


In [15]:
demographics.printSchema()

root
 |-- State: string (nullable = true)
 |-- StateCode: string (nullable = true)
 |-- City: string (nullable = true)
 |-- MedianAge: double (nullable = true)
 |-- MalePopulation: integer (nullable = true)
 |-- FemalePopulation: integer (nullable = true)
 |-- TotalPopulation: integer (nullable = true)
 |-- NumberofVeterans: integer (nullable = true)
 |-- ForeignBorn: integer (nullable = true)
 |-- AverageHouseholdSize: integer (nullable = true)
 |-- White: long (nullable = true)
 |-- Asian: long (nullable = true)
 |-- HispanicOrLatino: long (nullable = true)
 |-- AmericanIndianandAlaskaNative: long (nullable = true)
 |-- BlackOrAfricanAmerican: long (nullable = true)



In [16]:
demographics.show(1)

+-------+---------+-------------+---------+--------------+----------------+---------------+----------------+-----------+--------------------+-----+-----+----------------+-----------------------------+----------------------+
|  State|StateCode|         City|MedianAge|MalePopulation|FemalePopulation|TotalPopulation|NumberofVeterans|ForeignBorn|AverageHouseholdSize|White|Asian|HispanicOrLatino|AmericanIndianandAlaskaNative|BlackOrAfricanAmerican|
+-------+---------+-------------+---------+--------------+----------------+---------------+----------------+-----------+--------------------+-----+-----+----------------+-----------------------------+----------------------+
|Florida|       FL|Miami Gardens|     34.9|         50719|           62480|         113199|            2327|      33394|                   3|27273|    0|           23287|                            0|                 85300|
+-------+---------+-------------+---------+--------------+----------------+---------------+-------------

In [17]:
#Temparature 
fname = '../../data2/GlobalLandTemperaturesByCity.csv'
df_temperature =spark.read.csv(fname,header=True,sep=',',inferSchema='true')
df_temperature.printSchema()

root
 |-- dt: timestamp (nullable = true)
 |-- AverageTemperature: double (nullable = true)
 |-- AverageTemperatureUncertainty: double (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)



In [18]:
#Only filtering valid date and valid temperature
print("Total Temperature Records",df_temperature.count())
df_temperature.createOrReplaceTempView("temperature")
temperature = spark.sql(""" SELECT distinct *
                            FROM temperature 
                            where dt is not null 
                            and AverageTemperature is not null
                        """)
print("After eliminating invalid date and temperature",temperature.count())


Total Temperature Records 8599212
After eliminating invalid date and temperature 8235082


In [19]:
#Cost of living 
df_col =spark.read.csv('Cost of Living.csv',header=True,sep=',',inferSchema='true')

#Derived city code from city
df_col = df_col.withColumn("CityCode", substring_index(col("City"), ",", -1))
df_col = df_col.withColumn("City", substring_index(col("City"), ",", 1))
df_col.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- CostofLivingIndex: double (nullable = true)
 |-- RentIndex: double (nullable = true)
 |-- CostofLivingPlusRentIndex: double (nullable = true)
 |-- GroceriesIndex: double (nullable = true)
 |-- RestaurantPriceIndex: double (nullable = true)
 |-- LocalPurchasingPowerIndex: double (nullable = true)
 |-- CityCode: string (nullable = true)



In [20]:
df_col.count()

63

In [21]:
#Airport 
df_airport =spark.read.csv('airport-codes_csv.csv',header=True,sep=',',inferSchema='true')
df_airport.printSchema()

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: integer (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)



In [22]:
print("Total Record Count",df_airport.count())
df_airport.dropDuplicates()
df_airport = df_airport.withColumn("CityCode", substring_index(col("iso_region"), "-", -1))

Total Record Count 55075


In [23]:
df_airport.createOrReplaceTempView("airport")
airport = spark.sql(""" SELECT *
                            FROM airport 
                            where iso_country='US' and type <> 'closed'
                    """)
airport.show(1)

+-----+--------+-----------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+--------+
|ident|    type|             name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|CityCode|
+-----+--------+-----------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+--------+
|  00A|heliport|Total Rf Heliport|          11|       NA|         US|     US-PA|    Bensalem|     00A|     null|       00A|-74.9336013793945...|      PA|
+-----+--------+-----------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+--------+
only showing top 1 row



#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [24]:
#Basic checks on record count before writing as file or loading into table
print("Total Records in countries table",df_countries.count())
print("Total Records in visa table",df_visa.count())
print("Total Records in port table",df_port.count())
print("Total Records in immigrants table",immigrants.count())
print("Total Records in demographics table",demographics.count())
print("Total Records in temperature table",df_temperature.count())
print("Total Records in airline table",df_airline.count())
print("Total Records in cost of living table",df_col.count())
print("Total Records in airport table",airport.count())

Total Records in countries table 289
Total Records in visa table 90
Total Records in port table 660
Total Records in immigrants table 3254348
Total Records in demographics table 596
Total Records in temperature table 8599212
Total Records in airline table 61
Total Records in cost of living table 63
Total Records in airport table 21431


In [25]:
#Immigration Validation
filepath='stage/*.parquet'
df_immigrants = spark.read.parquet(filepath)
df_immigrants.createOrReplaceTempView("immigration")#print(all_files)
immigration_validation = spark.sql("""
                            SELECT  distinct i94mon 
                            FROM immigration
                            """)
immigration_validation.show()

+------+
|i94mon|
+------+
|   9.0|
+------+



In [ ]:
#writing output to parquet files
output_data='out/'
print("Writing back to parquet file...")
immigrants.write.mode("overwrite").parquet(output_data+"fact/")
df_countries.write.mode("overwrite").parquet(output_data+"country/")
df_visa.write.mode("overwrite").parquet(output_data+"visa/")
df_port.write.mode("overwrite").parquet(output_data+"port/")
demographics.write.mode("overwrite").parquet(output_data+"demographics/")
df_temperature.write.mode("overwrite").parquet(output_data+"temperature/")
df_airline.write.mode("overwrite").parquet(output_data+"airline/")
df_col.write.mode("overwrite").parquet(output_data+"costofliving/")
airport.write.mode("overwrite").parquet(output_data+"airport/")
print("Done")

#### 4.3 Data dictionary 
[Data Dictionary Link](https://r766466c839826xjupyterlnnfq3jud.udacity-student-workspaces.com/lab/tree/Data%20Dictionary.pdf).

#### Step 5: Complete Project Write Up

##### Tools and technologies
As explained earlier, we have selected option 2 for this project as tools and technologies involved are open source and no cost strategy

Option 2. Python, on-premise drives, Spark / Pandas, Juypter notebook

      Data Model    : Star Schema
      Cost Involve  : No
      Maintenance   : Difficult
      Scalability   : No
      Long Run      : Yes
      Over all Process: Process the data(Filtering, Cleansing, Transformation) using Spark then after processing, can save the data as Parquet files for Data Analysis

##### Propose how often the data should be updated and why      
* When considering the whole dataset used in this project, the data frequency should be monthly.

#### Possible Scenario's:
 * ###### In case the data was increased by 100x.
    We can go for the apporach 1, using Redshift, EMR cluster and Airflow would be able to manage scalabilty of data
 * ###### The data populates a dashboard that must be updated on a daily basis by 7am every day.
     By chance if we get daily, then we can schedule the airflow job to run prior to 5hrs of dashboard refresh 
 * ###### The database needed to be accessed by 100+ people.
    Yes we can make it by increasing red shift cluster instance 